In [1]:
import pandas as pd
import numpy as np

In [18]:
data="the movie is worst"

In [19]:
import re
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


ps=PorterStemmer()
all_stopwords=stopwords.words('english')
all_stopwords.remove('not')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
corpus=[]


review= re.sub('[^a-zA-Z]',' ', data )
review=review.lower()
review=review.split()
review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
review= ' '.join(review)
corpus.append(review)

In [21]:
corpus

['movi worst']

In [22]:
import pickle
path="E:\\downloads\\pickle\\bow.pkl"
cv= pickle.load(open(path,"rb"))

In [23]:
x_fresh=cv.transform(corpus).toarray()

In [24]:
x_fresh

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]], dtype=int64)

In [25]:
path="E:\\downloads\\pickle\\model.pkl"
model= pickle.load(open(path,"rb"))

In [26]:
prediction=model.predict(x_fresh)

In [27]:
prediction

array([2], dtype=int64)